# 회귀 기반 추천 시스템

> CatBoost(Categorical Booting)

In [15]:
!pip install catboost

In [16]:
# 깃에서 데이터 구조 확인 

In [17]:
import pandas as pd 

travel_df = pd.read_csv('./data/travel.csv')  # 범주형 데이터 셋
travel_df.shape

(34572, 15)

In [18]:
travel_df.head()

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,VISIT_AREA_NM,MVMN_NM,DGSTFN
0,남,30.0,1.0,4.0,2.0,2.0,6.0,2.0,2.0,7.0,3.0,3.0,미스틱3도,자가용,5.0
1,남,20.0,4.0,1.0,5.0,1.0,1.0,4.0,1.0,6.0,3.0,1.0,에스제이렌트카,대중교통 등,4.0
2,여,50.0,4.0,1.0,2.0,4.0,3.0,3.0,2.0,3.0,1.0,3.0,법환식당,대중교통 등,5.0
3,남,30.0,1.0,1.0,1.0,5.0,6.0,3.0,5.0,7.0,7.0,3.0,에코랜드호텔,자가용,5.0
4,여,20.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,1.0,2.0,윤스타피자앤파스타,자가용,4.0


In [19]:
travel_df.columns

Index(['GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM',
       'VISIT_AREA_NM', 'MVMN_NM', 'DGSTFN'],
      dtype='object')

In [ ]:
# 데이터 형태 변환
travel_df[['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']] = \
travel_df[['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']].astype(int)

In [21]:
travel_df.head()

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,VISIT_AREA_NM,MVMN_NM,DGSTFN
0,남,30,1,4,2,2,6,2,2,7,3,3,미스틱3도,자가용,5.0
1,남,20,4,1,5,1,1,4,1,6,3,1,에스제이렌트카,대중교통 등,4.0
2,여,50,4,1,2,4,3,3,2,3,1,3,법환식당,대중교통 등,5.0
3,남,30,1,1,1,5,6,3,5,7,7,3,에코랜드호텔,자가용,5.0
4,여,20,5,3,3,3,3,3,3,5,1,2,윤스타피자앤파스타,자가용,4.0


In [22]:
from sklearn.model_selection import train_test_split

X = travel_df.drop('DGSTFN', axis=1)
y = travel_df['DGSTFN']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from catboost import Pool  # 범주형 데이터 모델링 모듈

cat_festures = ['GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 'VISIT_AREA_NM', 'MVMN_NM']

train_pool = Pool(X_train, y_train, cat_features = cat_festures)  # cat_features= 모델 학습 시 범주형 데이터를 모델이 직접 이해해서 인코딩 처리 할 수 있도록 하는 파라미터
test_pool = Pool(X_test, y_test, cat_features = cat_festures)

In [26]:
from catboost import CatBoostRegressor

cb_reg = CatBoostRegressor(
    n_estimators = 500, 
    depth = 5, 
    learning_rate = 0.03, 
    loss_function = 'RMSE', 
    eval_metric = 'RMSE'
)

cb_reg.fit(train_pool, eval_set = test_pool, verbose=100)

0:	learn: 0.8341962	test: 0.8359310	best: 0.8359310 (0)	total: 212ms	remaining: 1m 45s
100:	learn: 0.7985342	test: 0.7931506	best: 0.7931506 (100)	total: 6.05s	remaining: 23.9s
200:	learn: 0.7863085	test: 0.7799891	best: 0.7799891 (200)	total: 11.5s	remaining: 17.1s
300:	learn: 0.7817498	test: 0.7763301	best: 0.7763301 (300)	total: 17.1s	remaining: 11.3s
400:	learn: 0.7787237	test: 0.7746257	best: 0.7746257 (400)	total: 22.1s	remaining: 5.47s
499:	learn: 0.7764294	test: 0.7733956	best: 0.7733956 (499)	total: 27.7s	remaining: 0us

bestTest = 0.773395606
bestIteration = 499



In [27]:
col_importance = pd.DataFrame({
    'column' : X_train.columns, 
    'importance' : cb_reg.feature_importances_
})

col_importance

,column,importance
0,GENDER,0.373389
1,AGE_GRP,7.351231
2,TRAVEL_STYL_1,7.058387
3,TRAVEL_STYL_2,6.958940
4,TRAVEL_STYL_3,4.288193
5,TRAVEL_STYL_4,9.604211
6,TRAVEL_STYL_5,8.382338
7,TRAVEL_STYL_6,8.360303
8,TRAVEL_STYL_7,8.541926
9,TRAVEL_STYL_8,11.937605


### 추천 시스템 구축
1. 방문지 목록을 생성
2. 사용자 특성 입력
3. 가상 만족도 예측
4. 만족도가 높은 순으로 추천

In [28]:
visit_areas = travel_df['VISIT_AREA_NM'].unique()
visit_areas.shape

(10711,)

In [ ]:
user_input = ['여', 60, 4, 4, 4, 4, 4, 4, 4, 4, 1, 2, '방문지', '자가용']
pred_results = []

for area in visit_areas : 
    user_input[-2] = area
    dgftfn_pred = cb_reg.predict(user_input)
    pred_results.append(dgftfn_pred)

pred_results[:10]   # 만족도 점수 반환

[np.float64(4.338881865504073),
 np.float64(4.149058377045566),
 np.float64(4.33214199958774),
 np.float64(4.190207715847474),
 np.float64(4.147286100972279),
 np.float64(4.126341464044342),
 np.float64(4.093713467767728),
 np.float64(4.147286100972279),
 np.float64(4.2713833543500925),
 np.float64(4.2051950289527)]

In [31]:
result_df = pd.DataFrame({
    'VISIT_AREA_NM' : visit_areas,
    'DGSTFN_PRED' : pred_results
})

result_df.sort_values(by='DGSTFN_PRED', ascending=False).head(10) # 만족도가 높은 방문지에 따라 10개 반환

,VISIT_AREA_NM,DGSTFN_PRED
47,보래드베이커스,4.555464
3054,대정오일시장,4.549863
2023,호텔연,4.548418
824,호텔리젠트마린,4.546289
1485,청파식당횟집,4.546285
4749,제주스럽닭 서귀포올레점,4.544025
6434,알뜨르비행장,4.536824
129,스누피가든,4.536047
3448,막둥이해녀복순이네,4.536003
481,하라케케,4.533105
